<h1><center>Лабораторна робота 6.</center></h1>
<h2><center>Лінійна регресія, Lasso і RF-регресія для задачі визначення якості вина</center></h2>

**Виконав:** Прізвище І.П.

**Варіант:** №__

<a class="anchor" id="lab-6"></a>

## Зміст

- [6.1. Завантаження та оброблення даних](#lab-6.1)
- [6.2. Лінійний регресійний аналіз](#lab-6.2)  
- [6.3. Регресійний аналіз із регуляризацією](#lab-6.3)
- [6.4. Побудова ансамблевих моделей](#lab-6.4)
- [6.5. Оцінювання та відбір ознак](#lab-6.5)
- [6.6. Покращення та подальший тюнинг моделей](#lab-6.6)

In [6]:
# відключимо попередження Anaconda
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split

<a class="anchor" id="lab-6.1"></a>

## <span style="color:blue; font-size:1.2em;">6.1. Завантаження та оброблення даних</span>

[Повернутися до змісту](#lab-6)

В цій лабораторній роботі будемо працювати з набором даних за якістю білого вина ([репозиторій UCI](https://archive.ics.uci.edu/ml/datasets/wine+quality)).

Набір даних "Wine Quality UCI" включає дані, що стосуються червоних і білих сортів португальського вина "Vinho Verde". Цей набір даних фокусується виключно на фізико-хімічних та сенсорних змінних, не включаючи інформацію про типи винограду, марки вина та ціни продажу. Wine Quality UCI може використовуватися для задач як класифікації, так і регресії.

Тут класи - впорядковані, але незбалансовані, що вказує на переважання нормальних вин над відмінними або поганими. Відповідність усіх вхідних змінних не є однозначною, що робить застосування методів відбору ознак до цієї задачі досить актуальним.

Набір даних складається з 11 неперервних вхідних змінних і однієї вихідної змінної:

1. Fixed Acidity (Фіксована кислотність) - відображає кількість нелетких кислот у вині.
2. Volatile Acidity (Летюча кислотність) - кількість оцтової кислоти у вині, яка при занадто високому рівні може призвести до неприємного, схожого на оцет, смаку.
3. Citric Acid (лимонна кислота) - у невеликих кількостях лимонна кислота може додавати вину "свіжості" та аромату.
4. Residual Sugar (залишковий цукор) - кількість цукру, що залишається після припинення ферментації, рідко можна зустріти вина з вмістом цукру менше 1 грама/літр, а вина з вмістом цукру понад 45 грамів/літр вважаються солодкими.
5. Chlorides (хлориди) - кількість солі у вині.
6. Free Sulfur Dioxide (вільний діоксид сірки) - вільна форма SO2 існує в рівновазі між молекулярним SO2 (у вигляді розчиненого газу) і бісульфіт-іоном; він запобігає розмноженню мікроорганізмів і окисленню вина.
7. Total Sulfur Dioxide (загальний діоксид сірки) - сума вільної та зв'язаної форм S02; у низьких концентраціях SO2 майже не відчувається у вині, але при концентрації вільного SO2 понад 50 проміле, SO2 стає помітним у носі та смаку вина.
8. Density (щільність) - густина вина близька до густини води і залежить від вмісту алкоголю і цукру.
9. pH - описує кислотність або лужність вина за шкалою від 0 (дуже кисле) до 14 (дуже лужне); більшість вин знаходиться між 3-4 балами за шкалою pH.
10. Sulphates (сульфати) - добавка до вина, яка може сприяти підвищенню рівня діоксиду сірки (S02), що діє як антимікробний засіб і антиоксидант.
11. Alcohol (алкоголь) - відсотковий вміст алкоголю у вині.
12. Quality (якість) - вихідна змінна (цільова змінна), що базується на сенсорних даних і оцінюється від 0 до 10 балів.


Детальний опис цього набору даних наведено у науковій роботі [Using Data Mining for Wine Quality Assessment](https://www.researchgate.net/publication/221612614_Using_Data_Mining_for_Wine_Quality_Assessment)

In [7]:
winequality_white_url = 'https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/refs/heads/main/02_assignments/ida_lab-06_regression-wine/winequality-white.csv'
data = pd.read_csv(winequality_white_url, sep=';')

In [8]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


Виконаємо відокремлення цільової ознаки. Для цього розділимо навчальну вибірку за відношенням 7:3; тут 30% даних - тестова вибірка. Задамо `random_state=17` і виконаємо стандартизацію даних за допомогою `StandardScaler()`. Далі будемо працювати зі змінними `X_train_scaled` та `X_test_scaled`.

In [10]:
y = data['quality']
X = data.drop('quality', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

<a class="anchor" id="lab-6.2"></a>

## <span style="color:blue; font-size:1.2em;">6.2. Лінійний регресійний аналіз</span>

[Повернутися до змісту](#lab-6)

### <span style="color:red; font-size:1.5em;">Завдання 1</span>

---

**Для всіх варіантів:**

**Побудуйте модель простої лінійної регресії за вхідними даними оцінок якості білого вина. Майте на увазі, що для навчання та тестування моделі треба використати стандартизовані значення незалежних змінних `X_train_scaled` та `X_test_scaled`, і початкові значення цільової ознаки `y_train` та `y_test`.**

---

**Варіант 1:** Додайте до побудованої лінійної моделі поліноміальні ознаки з допомогою `PolynomialFeatures` (степені 2 і 3). Проаналізуйте, як додавання цих ознак впливає на продуктивність і стабільність моделі. Використайте регуляризацію через `Ridge`, щоби зменшити перенавчання.

*Технічна примітка:* Застосуйте `PolynomialFeatures(degree=2)` та `PolynomialFeatures(degree=3)` з бібліотеки `sklearn.preprocessing`. Для регуляризації `Ridge` з `alpha=1.0` з `sklearn.linear_model`.

---

**Варіант 2:** Використовуючи просту лінійну регресію, дослідіть вплив різних методів масштабування на продуктивність моделі. Порівняйте результати, отримані після оброблення даних за допомогою `StandardScaler` та `MinMaxScaler`. Оцініть модель за допомогою RMSE та R², щоби зрозуміти, як масштабування впливає на якість передбачень та на стабільність моделі.

*Технічна примітка:* Використовуйте `StandardScaler()` та `MinMaxScaler()` з `sklearn.preprocessing`, метрики `mean_squared_error` та `r2_score` з `sklearn.metrics`.

---

**Варіант 3:** Реалізуйте покрокову регресію з комбінацією прямого та зворотного відбору ознак для моделі лінійної регресії. Використовуйте `RFE` для автоматизованого виключення найменш значущих ознак і додайте ознаки вручну на кожному кроці. Також на кожному кроці оцінюйте точність та стабільність моделі.

*Технічна примітка:* Застосуйте `RFE(estimator=LinearRegression())` з `sklearn.feature_selection`. Використовуйте `n_features_to_select=5` для обмеження кількості відібраних ознак.

---

**Варіант 4:** Створіть ще одну модель лінійної регресії з регуляризацією ElasticNet, досліджуючи компроміс між Lasso та Ridge штрафами. Підберіть параметр `l1_ratio`, змінюючи його від 0.1 до 0.9, щоби побачити, як різні пропорції штрафів впливають на точність моделі та на значення коефіцієнтів ознак.

*Технічна примітка:* Застосуйте `ElasticNet` з параметрами `l1_ratio=[0.1, 0.5, 0.9]` з `sklearn.linear_model`. Використовуйте функцію `GridSearchCV` для підбору оптимального `alpha`.

---

**Варіант 5:** Проаналізуйте вплив логарифмічного перетворення викривлених ознак на продуктивність лінійної регресії. Використовуйте `numpy.log1p` для перетворення і побудуйте модель, порівнявши її з початковою для оцінювання змін у точності та стабільності.

*Технічна примітка:* Використовуйте `numpy.log1p(data['feature'])` для перетворення ознак із викривленим розподілом, після чого побудуйте модель лінійної регресії.

---

**Варіант 6:** Використайте `PolynomialFeatures` з параметром `interaction_only=True`, щоб створити нові ознаки на основі взаємодії між наявними ознаками. Проаналізуйте, як додавання взаємодій впливає на точність і стабільність моделі, та виявіть найбільш значущі з них.

*Технічна примітка:* Застосуйте `PolynomialFeatures(interaction_only=True)` з `sklearn.preprocessing`, після чого виконайте лінійну регресію.

---

**Варіант 7:** Застосуйте `QuantileTransformer` для трансформації ознак у вигляді нормального розподілу й побудуйте лінійну регресію. Оцініть, як така трансформація впливає на продуктивність моделі, порівнюючи її зі звичайною моделлю.

*Технічна примітка:* Використовуйте `QuantileTransformer(output_distribution='normal')` з `sklearn.preprocessing`.

---

**Варіант 8:** Використовуйте `RANSACRegressor` для побудови моделі надійної регресії, щоби зменшити вплив пропусків на кінцевий результат. Порівняйте продуктивність `RANSACRegressor` з результатами простої лінійної регресії та оцініть відмінності.

*Технічна примітка:* Застосуйте `RANSACRegressor(base_estimator=LinearRegression())` з `sklearn.linear_model`. Використовуйте параметр `min_samples=50` для стабільності моделі.

---

**Варіант 9:** Використайте `LassoCV` для автоматичного відбору ознак, що поєднує регресію Lasso з крос-валідацією. Це дасть змогу виявити найбільш значущі ознаки та запобігти перенавчанню. Оцініть якість моделі після такого відбору.

*Технічна примітка:* Використовуйте `LassoCV(cv=5)` з `sklearn.linear_model` та відсортуйте значущі ознаки за коефіцієнтами.

---

**Варіант 10:** Після побудови моделі лінійної регресії перегляньте її коефіцієнти та відсортуйте ознаки за впливом на цільову ознаку якості вина. Розгляньте великі за модулем значення коефіцієнтів, включно з негативними, як індикатори сильного впливу.

*Технічна примітка:* Використовуйте `LinearRegression().coef_` для перегляду коефіцієнтів моделі та `numpy.argsort` для сортування.

---

**Варіант 11:** Використайте метод відбору ознак на основі інформаційного критерію `mutual_info_regression` для вибору найбільш інформативних ознак. Побудуйте модель лінійної регресії з відібраними ознаками та порівняйте результати із моделлю, де використовуються всі доступні ознаки.

*Технічна примітка:* Застосуйте `mutual_info_regression` з `sklearn.feature_selection` разом із `SelectKBest(score_func=mutual_info_regression, k=5)`.

---

**Варіант 12:** Порівняйте продуктивність моделі лінійної регресії за використання `RobustScaler` замість стандартного масштабування. Оцініть, як впливає така зміна на стійкість моделі щодо відхилень.

*Технічна примітка:* Використовуйте `RobustScaler()` з `sklearn.preprocessing` перед побудовою моделі.

---

**Варіант 13:** Дослідіть вплив різних значень `alpha` у моделі Ridge на точність та стабільність регресії. Виберіть оптимальний `alpha` через `GridSearchCV`.

*Технічна примітка:* Використовуйте `Ridge()` з `alpha` у діапазоні `[0.01, 0.05, 0.1, 1, 10]` і `GridSearchCV(cv=5)`.

---

**Варіант 14:** Створіть ансамблеву модель регресії, використовуючи `BaggingRegressor` із базовою лінійною регресією. Проаналізуйте, як така модель покращує точність та зменшує вплив пропусків.

*Технічна примітка:* Використовуйте `BaggingRegressor(base_estimator=LinearRegression(), n_estimators=50)` з `sklearn.ensemble`.

---

**Варіант 15:** Використайте `HuberRegressor` для оброблення даних із можливими викидами. Цей метод поєднує стандартну лінійну регресію з підходом, стійким до викидів. Порівняйте продуктивність `HuberRegressor` зі звичайною лінійною регресією.

*Технічна примітка:* Використовуйте `HuberRegressor(epsilon=1.35)` з `sklearn.linear_model`.

---

**Варіант 16:** Проведіть аналіз впливу різних порогових значень у функції відбору ознак SelectFromModel, застосовуючи її до моделі лінійної регресії з регуляризацією Lasso. Порівняйте результати продуктивності залежно від вибраних ознак.

*Технічна примітка:* Використовуйте `SelectFromModel(estimator=Lasso(alpha=0.01))` з `sklearn.feature_selection`.

---

**Варіант 17:** Використайте `SelectKBest` для відбору найкращих ознак на основі статистичної міри. Застосуйте тест `f_regression`, щоби відібрати ознаки з найсильнішим впливом на цільову змінну, і побудуйте модель лінійної регресії, використовуючи тільки відібрані ознаки. Порівняйте продуктивність моделі до й після відбору ознак.

*Технічна примітка:* Використовуйте `SelectKBest(score_func=f_regression, k=5)` з `sklearn.feature_selection`.

---

**Варіант 18:** Проведіть відбір ознак на основі важливості за допомогою `RandomForestRegressor`, а потім використайте ці важливі ознаки для побудови лінійної регресійної моделі. Дослідіть, як вибір лише найбільш значущих ознак із моделі випадкового лісу впливає на точність та стабільність лінійної регресії.

*Технічна примітка:* Використовуйте `RandomForestRegressor()` з `sklearn.ensemble` для визначення важливості ознак та `SelectFromModel` для їх відбору.

---

**Варіант 19:** Використайте `Recursive Feature Elimination with Cross-Validation (RFECV)` для автоматичного визначення оптимальної кількості ознак. Побудуйте лінійну регресійну модель із відібраними ознаками та оцініть її продуктивність, порівнюючи її з повною моделлю, щоби побачити, чи зміниться точність при відборі ознак.

*Технічна примітка:* Застосуйте `RFECV(estimator=LinearRegression(), cv=5)` з `sklearn.feature_selection`.

---

**Варіант 20:** Використайте `L1-базований відбір ознак`, застосувавши Lasso регресію для видалення ознак із незначним впливом. Порівняйте продуктивність моделі до та після видалення ознак. Цей метод допоможе зменшити розмірність моделі та покращити її інтерпретованість.

*Технічна примітка:* Використовуйте `Lasso(alpha=0.01, max_iter=10000)` з `sklearn.linear_model` і `SelectFromModel` для відбору значущих ознак.

<a class="anchor" id="lab-6.3"></a>

## <span style="color:blue; font-size:1.2em;">6.3. Регресійний аналіз із регуляризацією</span>

[Повернутися до змісту](#lab-6)

### <span style="color:red; font-size:1.5em;">Завдання 2</span>

---

**Для всіх варіантів:**

**Побудуйте моделі Lasso-регресі з параметрами `alpha=0.01`, `random_state=17` та `cv=5`. Обрахуйте вплив усіх незалежних ознак на цільову ознаку Lasso-регресії. Відсортуйте ознаки за впливом на якість вина. Майте на увазі, що для навчання та тестування моделі треба використати стандартизовані значення незалежних змінних `X_train_scaled` та `X_test_scaled`, і початкові значення цільової ознаки `y_train` та `y_test`.**

---

**Варіант 1:** Виконайте відбір ознак на основі їхньої важливості для Lasso-регресії. Після побудови моделі за допомогою Lasso проаналізуйте коефіцієнти отриманих ознак: залиште тільки ті, що мають ненульові значення. Потім порівняйте продуктивність отриманої моделі з моделлю, що використовує всі ознаки, та оцініть, наскільки вибір основних ознак впливає на точність передбачень.

*Технічна примітка:* Використайте `Lasso(alpha=0.01)` для отримання важливих ознак та `SelectFromModel` з `sklearn.feature_selection`, щоб залишити тільки ті, які мають значний вплив.

---

**Варіант 2:** Після побудови моделі Lasso-регресії оберіть ознаки за допомогою методу `SelectKBest`, що базується на статистичній значущості кожної ознаки. Проаналізуйте, як вибір найвпливовіших ознак на основі їхніх значень у моделі впливає на точність прогнозування. Це допоможе визначити оптимальну кількість ознак для передбачення цільової змінної.

*Технічна примітка:* Використайте `SelectKBest(score_func=f_regression, k=5)` з `sklearn.feature_selection` для відбору ознак на основі значень Lasso.

---

**Варіант 3:** Застосуйте метод зворотного виключення ознак (RFE) для побудови Lasso-регресії. Методично видаляйте ознаки з найменшим впливом, поки не залишаться тільки найбільш важливі. Порівняйте результати продуктивності моделі зі зворотним виключенням і без нього, щоби зрозуміти, як відбір ознак впливає на передбачення.

*Технічна примітка:* Використайте `RFE(estimator=Lasso(alpha=0.01))` з `sklearn.feature_selection` для покрокового видалення ознак.

---

**Варіант 4:** Проведіть груповий аналіз ознак для відбору найбільш впливових груп за допомогою GroupLasso. Створіть модель, у якій деякі ознаки об’єднані в групи. Порівняйте, як модель обробляє ці групи порівняно з окремими ознаками, та оцініть, чи сприяє групування покращенню передбачуваності.

*Технічна примітка:* Для реалізації Використайте `group-lasso` з груповими ознаками або `GroupLassoCV`, щоби спростити відбір груп ознак.

---

**Варіант 5:** Дослідіть вплив кожної ознаки на цільову змінну за допомогою побудови Lasso-регресії з різними значеннями параметра `alpha`. Оцініть, як змінюється кількість ознак із ненульовими коефіцієнтами у випадку збільшення `alpha`, та визначте оптимальний рівень регуляризації для відбору найбільш впливових ознак.

*Технічна примітка:* Застосуйте `LassoCV` з параметром `alphas=np.linspace(0.01, 1.0, 10)` для пошуку оптимального значення регуляризації.

---

**Варіант 6:** Виконайте відбір ознак на основі важливості за допомогою випадкового лісу, а потім використайте відібрані ознаки для побудови моделі Lasso-регресії. Оцініть, як поєднання випадкового лісу для відбору та Lasso-регресії для моделювання впливає на точність передбачень і стабільність моделі.

*Технічна примітка:* Використайте `RandomForestRegressor()` для обчислення важливості ознак та `SelectFromModel` для їхнього відбору перед застосуванням `Lasso(alpha=0.01)`.

---

**Варіант 7:** Дослідіть вплив відбору ознак на основі кореляційного аналізу, залишивши лише ті ознаки, які мають високий кореляційний зв’язок із цільовою змінною. Потім побудуйте модель Lasso-регресії з відфільтрованими ознаками та порівняйте її результати з моделлю, що використовує всі ознаки.

*Технічна примітка:* Використайте `data.corr()` з бібліотеки `pandas` для обчислення кореляції, щоби залишити тільки значущі ознаки перед застосуванням `Lasso(alpha=0.01)`.

---

**Варіант 8:** Використайте метод відбору ознак `mutual_info_regression` для аналізу взаємної інформації кожної ознаки із цільовою змінною. Виберіть тільки найбільш інформативні ознаки, а потім побудуйте модель Lasso-регресії, щоб оцінити, як використання обраних ознак впливає на передбачення.

*Технічна примітка:* Використайте `mutual_info_regression` з `sklearn.feature_selection` та `SelectKBest(score_func=mutual_info_regression, k=5)`.

---

**Варіант 9:** Після побудови моделі Lasso-регресії перегляньте її коефіцієнти та відсортуйте ознаки за їхнім впливом на цільову змінну. Використайте великі за модулем значення коефіцієнтів для визначення найважливіших ознак. Створіть візуалізацію впливу ознак за допомогою графіка стовпців.

*Технічна примітка:* Використайте `LinearRegression().coef_` для перегляду коефіцієнтів і `matplotlib.pyplot.bar` для візуалізації впливу ознак.

---

**Варіант 10:** Дослідіть, як стабільність ознак змінюється в процесі регуляризації Lasso-регресії, створюючи кілька бутстреп-вибірок початкового набору даних. Перегляньте, які ознаки постійно залишаються значущими у всіх вибірках, щоби виділити найстабільніші ознаки для моделювання.

*Технічна примітка:* Використайте функцію `resample` з `sklearn.utils` для створення бутстреп-вибірок та `Lasso(alpha=0.01)` для моделювання впливу ознак.

---

**Варіант 11:** Використайте метод головних компонент (PCA) для відбору найбільш значущих ознак та побудуйте модель Lasso-регресії на зменшеному наборі ознак. Проаналізуйте, як зменшення кількості ознак за допомогою PCA впливає на точність передбачення та на інтерпретованість моделі.

*Технічна примітка:* Використайте `PCA(n_components=5)` з `sklearn.decomposition` для відбору основних компонент перед застосуванням `Lasso(alpha=0.01)`.

---

**Варіант 12:** Використайте Lasso-регресію для аналізу взаємодій між ознаками, поєднуючи її з методом взаємної інформації. Спочатку визначте найбільш інформативні ознаки за допомогою `mutual_info_regression`, а потім застосуйте Lasso, щоби проаналізувати вплив взаємодій між обраними ознаками на точність передбачення.

*Технічна примітка:* Використайте `mutual_info_regression` з `SelectKBest` для попереднього відбору ознак, а потім `PolynomialFeatures(interaction_only=True)` з `Lasso(alpha=0.01)`.

---

**Варіант 13:** Виконайте відбір ознак методом послідовного додавання (forward selection) для побудови моделі Lasso-регресії. Додавайте ознаки почергово, оцінюючи їхній вплив на точність моделі. Цей підхід дасть змогу визначити оптимальний набір ознак, які найбільше впливають на передбачення цільової змінної.

*Технічна примітка:* Використайте `SequentialFeatureSelector` з параметром `direction='forward'` з `sklearn.feature_selection`, а потім застосуйте `Lasso(alpha=0.01)` для моделювання.

---

**Варіант 14:** Проведіть експеримент з Lasso-регресією, використовуючи різні вагові коефіцієнти для кожної ознаки. Задайте вищі ваги для ознак, які є найбільш значущими за попереднім аналізом, й оцініть, як це впливає на продуктивність моделі та вибір основних ознак.

*Технічна примітка:* Використайте параметр `sample_weight` в `Lasso(alpha=0.01)` з `sklearn.linear_model` для коригування вагових коефіцієнтів ознак.

---

**Варіант 15:** Побудуйте модель Lasso-регресії з урахуванням зворотного виключення ознак на основі розрахунку їхньої інформаційної значущості. Видаляйте ознаки, які найменш інформативні на кожному етапі, та залиште лише найбільш значущі для фінальної моделі. Оцініть стабільність та точність отриманих результатів.

*Технічна примітка:* Використайте `RFE(estimator=Lasso(alpha=0.01))` для поетапного видалення ознак та `mutual_info_regression` для визначення їх значущості.

---

**Варіант 16:** Застосуйте багаторазовий метод відбору ознак на основі випадкових вибірок. Згенеруйте кілька випадкових вибірок ознак і побудуйте модель Lasso-регресії для кожної з них. Оцініть, які ознаки найбільш стійко обираються в різних вибірках, і залиште їх для остаточної моделі.

*Технічна примітка:* Використайте `RandomizedSearchCV` з `Lasso(alpha=0.01)` для створення випадкових вибірок ознак.

---

**Варіант 17:** Використайте метод `SelectFdr` (False Discovery Rate) для відбору ознак, які найбільш значущо корелюють із цільовою змінною. Після обрання ознак побудуйте модель Lasso-регресії та проаналізуйте, як відбір за значущістю впливає на точність передбачень.

*Технічна примітка:* Використайте `SelectFdr(score_func=f_regression, alpha=0.05)` з `sklearn.feature_selection` для відбору ознак, а потім застосуйте `Lasso(alpha=0.01)`.

---

**Варіант 18:** Побудуйте модель Lasso-регресії з крос-валідацією для оцінювання точності та стабільності відбору ознак. Використайте `cross_val_score` для обрахунку середніх показників точності на кожному етапі, щоби виявити, які ознаки залишаються найбільш значущими незалежно від розподілу даних.

*Технічна примітка:* Використайте `cross_val_score(Lasso(alpha=0.01), X, y, cv=5)` з `sklearn.model_selection` для проведення крос-валідації.

---

**Варіант 19:** Виконайте відбір ознак за допомогою методу «моделювання ансамблів», де декілька моделей (Lasso, Ridge та Decision Tree) обирають найбільш значущі ознаки. Проаналізуйте, які ознаки стабільно залишаються значущими в різних моделях, та використайте їх у моделі Lasso-регресії для точного передбачення.

*Технічна примітка:* Використайте `VotingRegressor([('lasso', Lasso(alpha=0.01)), ('ridge', Ridge()), ('tree', DecisionTreeRegressor())])` з `sklearn.ensemble`.

---

**Варіант 20:** Застосуйте адаптивну Lasso-регресію з ваговими коефіцієнтами для кожної ознаки на основі їхньої початкової важливості. Визначте початкові коефіцієнти, використовуючи іншу регресійну модель, а потім скоригуйте ваги для Lasso, щоби зробити модель більш гнучкою до впливових ознак.

*Технічна примітка:* Використайте `Lasso(alpha=0.01)` для початкового відбору ознак і `sample_weight` для вагових коефіцієнтів під час побудови остаточної адаптивної Lasso-регресії.

<a class="anchor" id="lab-6.4"></a>

## <span style="color:blue; font-size:1.2em;">6.4. Побудова ансамблевих моделей</span>

[Повернутися до змісту](#lab-6)

### <span style="color:red; font-size:1.5em;">Завдання 3</span>

---

**Для всіх варіантів:**

**Побудуйте модель випадкового лісу з гіперпараметрами за замовчуванням. Для навчання та тестування моделі треба використати стандартизовані значення незалежних змінних `X_train_scaled` та `X_test_scaled`, і початкові значення цільової ознаки `y_train` та `y_test`.**

---

**Варіант 1:** Використайте `GridSearchCV` для ретельного налаштування гіперпараметрів моделі випадкового лісу. Дослідіть такі параметри, як `n_estimators` (кількість дерев), `max_depth` (максимальна глибина дерева) та `min_samples_split` (мінімальна кількість зразків для розбиття вузла). Оцініть, як кожен параметр впливає на точність моделі для визначення якості вина.

*Технічна примітка:* Використайте `GridSearchCV` з параметром `cv=5` для налаштування крос-валідації та параметрами діапазону для `n_estimators=[50, 100, 150]`, `max_depth=[10, 20, None]`, `min_samples_split=[2, 5, 10]` у `sklearn.ensemble.RandomForestClassifier`.

---

**Варіант 2:** Побудуйте та порівняйте дві моделі — Random Forest та Extra Trees. Проаналізуйте, як кожен з ансамблевих методів (беггінг у випадковому лісі та випадковий розщеп в Extra Trees) впливає на загальну результативність і стабільність моделей. Зробіть висновки щодо того, яка модель краще підходить для даного набору даних.

*Технічна примітка:* Використайте `RandomForestRegressor` та `ExtraTreesRegressor` з `sklearn.ensemble`, порівнюючи метрики `mean_squared_error` та `r2_score`.

---

**Варіант 3:** Застосуйте моделі беггінгу для логістичної регресії та дерев рішень, а також випадковий ліс, щоби порівняти їхні результати на основі метрики AUC ROC. Визначте, яка з моделей має найкращу здатність відокремлювати класи для даних про якість вина.

*Технічна примітка:* Використайте `BaggingClassifier` з базовими моделями `LogisticRegression` і `DecisionTreeClassifier`, а також `RandomForestClassifier` з `roc_auc_score` з `sklearn.metrics`.

---

**Варіант 4:** Для оброблення незбалансованих класів у наборі даних Використайте `BalancedRandomForestClassifier` з бібліотеки `imbalanced-learn`. Оцініть вплив автоматичного балансування на результативність моделі та порівняйте його з результатами звичайного випадкового лісу.

*Технічна примітка:* Використайте `BalancedRandomForestClassifier` з `imblearn.ensemble` та `RandomForestClassifier` з `class_weight='balanced'` для порівняння результативності.

---

**Варіант 5:** Проведіть дослідження на основі різних значень глибини дерев у моделі випадкового лісу. Виконайте експерименти зі значеннями `max_depth` від 5 до 30 і оцініть, як різні рівні глибини впливають на точність, обчислювальну складність та узагальненість моделі.

*Технічна примітка:* Застосуйте параметр `max_depth` зі значеннями `[5, 10, 20, 30, None]` у `RandomForestClassifier` або `RandomForestRegressor`, використовуючи метрики `accuracy_score` або `mean_squared_error`.

---

**Варіант 6:** Дослідіть, як вибір критерію розбиття вузлів у випадковому лісі (наприклад, `gini` та `entropy`) впливає на результативність моделі. Застосуйте обидва критерії, побудуйте дві моделі та порівняйте їх за метриками точності, щоби визначити, який критерій підходить краще.

*Технічна примітка:* Використайте параметр `criterion='gini'` та `criterion='entropy'` з `RandomForestClassifier`, порівнюючи точність за допомогою `accuracy_score` з `sklearn.metrics`.

---

**Варіант 7:** Перед побудовою моделі випадкового лісу виконайте відбір ознак за допомогою `SelectFromModel` із застосуванням Lasso або `Recursive Feature Elimination (RFE)`. Оцініть, як обрані ознаки впливають на результативність моделі, і порівняйте точність із моделлю, побудованою на повному наборі ознак.

*Технічна примітка:* Використайте `SelectFromModel` з `Lasso(alpha=0.01)` або `RFE(estimator=RandomForestClassifier())` з `sklearn.feature_selection`.

---

**Варіант 8:** Експериментуйте з різними стратегіями вибірки, такими як `bootstrap=True` та `bootstrap=False`, під час створення дерев у випадковому лісі. Проаналізуйте, як ці стратегії впливають на зміщення та дисперсію моделі, а також на її здатність до узагальнення.

*Технічна примітка:* Використайте параметр `bootstrap=True` та `bootstrap=False` у `RandomForestClassifier` та порівняйте метрики `accuracy_score` і `f1_score`.

---

**Варіант 9:** Побудуйте моделі беггінгу для логістичної регресії та дерев рішень, порівнюючи їх із випадковим лісом за допомогою показників середньоквадратичного відхилення точності. Це дасть змогу оцінити, яка з моделей має стабільнішу результативність.

*Технічна примітка:* Використайте `BaggingClassifier` з `LogisticRegression` та `DecisionTreeClassifier`, а також `RandomForestClassifier`. Розрахуйте середньоквадратичне відхилення точності через `cross_val_score`.

---

**Варіант 10:** Використайте модель випадкового лісу з балансуванням ваги класу, налаштувавши параметр `class_weight='balanced'`. Порівняйте результативність моделі з і без bootstrap-вибірок, щоби побачити вплив балансування та різних стратегій вибірки на точність передбачень.

*Технічна примітка:* Використайте параметр `class_weight='balanced'` у `RandomForestClassifier` з `bootstrap=True` та `bootstrap=False`, і порівняйте результати за допомогою `roc_auc_score` та `accuracy_score`.

**Варіант 11:** Виконайте налаштування гіперпараметрів моделі випадкового лісу з використанням `RandomizedSearchCV`. Дослідіть широкий діапазон значень параметрів `n_estimators`, `max_features`, `min_samples_leaf` та `max_depth`, щоб знайти оптимальні налаштування для найкращої результативності моделі на завданні класифікації якості вина.

*Технічна примітка:* Використайте `RandomizedSearchCV` з параметрами `n_iter=50` та `cv=5` для вибору випадкових гіперпараметрів. Для налаштування Використайте `n_estimators=[50, 100, 150, 200]`, `max_features=['auto', 'sqrt']`, `min_samples_leaf=[1, 2, 4]`, та `max_depth=[10, 20, 30, None]`.

---

**Варіант 12:** Порівняйте випадковий ліс із градієнтним бустингом (`GradientBoostingClassifier`) для завдання класифікації якості вина. Проведіть аналіз на основі метрик точності та обчислювальної ефективності, щоб зрозуміти переваги та недоліки кожного методу для даного набору даних.

*Технічна примітка:* Використайте `RandomForestClassifier` і `GradientBoostingClassifier` з `sklearn.ensemble`, оцінюючи моделі за допомогою `accuracy_score` та `f1_score`.

---

**Варіант 13:** Використайте метод `SelectKBest` для відбору ознак перед побудовою моделі випадкового лісу. Досліджуйте, як зменшення кількості ознак на основі статистичної значущості впливає на результативність моделі та її узагальнення.

*Технічна примітка:* Використайте `SelectKBest(score_func=f_classif, k=10)` з `sklearn.feature_selection` для відбору ознак і `RandomForestClassifier` для побудови моделі.

---

**Варіант 14:** Виконайте аналіз стабільності моделі випадкового лісу, застосовуючи крос-валідацію з різними значеннями `random_state`. Оцініть, як зміни в початковому випадковому стані впливають на стабільність точності та структуру дерев у моделі.

*Технічна примітка:* Використайте `cross_val_score` з різними значеннями `random_state` у `RandomForestClassifier`, наприклад `[0, 42, 99, 123]`, та оцінюйте середнє значення точності.

---

**Варіант 15:** Проведіть експеримент із частковим ансамблюванням у випадковому лісі, використовуючи лише частину ознак під час побудови кожного дерева (`max_features`). Оцініть, як варіація значень `max_features` впливає на результативність моделі та її обчислювальну ефективність.

*Технічна примітка:* Використайте `RandomForestClassifier` із параметром `max_features=[0.3, 0.5, 'auto', 'sqrt']` для оцінювання впливу часткового ансамблювання.

---

**Варіант 16:** Дослідіть вплив регуляризації моделі випадкового лісу, налаштувавши параметри `min_samples_split` та `min_samples_leaf`. Проведіть аналіз, як збільшення значень цих параметрів впливає на узагальненість моделі, її точність та складність структури дерев.

*Технічна примітка:* Використайте `RandomForestClassifier` із параметрами `min_samples_split=[2, 5, 10]` та `min_samples_leaf=[1, 2, 4]`, оцінюючи точність за допомогою `accuracy_score`.

---

**Варіант 17:** Використайте `Feature Importance` з випадкового лісу для визначення найбільш впливових ознак у наборі даних. Після визначення ключових ознак побудуйте модель тільки на основі цих ознак і порівняйте результативність із повною моделлю.

*Технічна примітка:* Використайте атрибут `feature_importances_` у `RandomForestClassifier` для визначення важливості ознак та `SelectFromModel` для вибору найбільш значущих ознак.

---

**Варіант 18:** Виконайте кластеризацію ознак за допомогою `FeatureAgglomeration` перед побудовою випадкового лісу. Створіть менший набір ознак на основі кластерів і проаналізуйте, як це впливає на результативність і швидкість моделі.

*Технічна примітка:* Використайте `FeatureAgglomeration` з `sklearn.cluster` для зменшення кількості ознак перед застосуванням `RandomForestClassifier`.

---

**Варіант 19:** Використайте `Stochastic Gradient Boosting (SGD)` разом із випадковим лісом, щоби побудувати модель гібридного ансамблю. Проаналізуйте, як цей підхід впливає на точність та узагальнення проти класичних випадкових лісів.

*Технічна примітка:* Використайте `VotingClassifier` з моделями `RandomForestClassifier` та `SGDClassifier` з параметром `voting='soft'` з `sklearn.ensemble`.

---

**Варіант 20:** Застосуйте стратегію розширення ознак, додаючи до початкових ознак статистичні показники, як-от середнє, стандартне відхилення та медіану. Побудуйте модель випадкового лісу з розширеним набором ознак і оцініть, чи допомагає додаткова інформація поліпшити точність передбачень.

*Технічна примітка:* Використайте функції `mean()`, `std()` та `median()` з `pandas` для створення нових ознак, потім застосуйте `RandomForestClassifier` для моделювання.

<a class="anchor" id="lab-6.5"></a>

## <span style="color:blue; font-size:1.2em;">6.5. Оцінювання та відбір ознак</span>

[Повернутися до змісту](#lab-6)

### <span style="color:red; font-size:1.5em;">Завдання 4</span>

---

**Варіант 1:** Використайте `Principal Component Analysis (PCA)` для зменшення розмірності набору ознак перед побудовою моделі випадкового лісу. Перетворіть початкові ознаки на основні компоненти та оберіть перші компоненти, що пояснюють значну частину дисперсії (>90 %). Порівняйте результативність моделі, побудованої на основних компонентах, з моделлю, побудованою на початкових ознаках.

*Технічна примітка:* Використайте `PCA(n_components=10)` з `sklearn.decomposition` для створення основних компонент і `RandomForestClassifier` для побудови моделі.

---

**Варіант 2:** Виконайте `Linear Discriminant Analysis (LDA)` для зменшення розмірності ознак, зберігаючи інформацію про класи. Виберіть найбільш дискримінуючі компоненти для побудови моделі випадкового лісу та оцініть, як використання LDA впливає на точність передбачення порівняно з повним набором ознак.

*Технічна примітка:* Застосуйте `LinearDiscriminantAnalysis(n_components=2)` з `sklearn.discriminant_analysis` до вибору найбільш значущих компонент і Використайте `RandomForestClassifier` для моделювання.

---

**Варіант 3:** Застосуйте метод `Recursive Feature Addition` до побудови моделі випадкового лісу з поступовим додаванням ознак. На кожному етапі додавайте ознаку, яка найбільше покращує результативність моделі, і продовжуйте, допоки не досягнете бажаного рівня точності. Оцініть остаточний набір обраних ознак.

*Технічна примітка:* Використайте `SequentialFeatureSelector` з параметром `direction='forward'` у `sklearn.feature_selection`, а потім `RandomForestClassifier` для побудови остаточної моделі.

---

**Варіант 4:** Здійсніть відбір ознак на основі взаємного інформаційного індексу, щоб відібрати ознаки з найвищою мірою залежності із цільовою змінною. Оберіть найінформативніші ознаки для побудови моделі випадкового лісу й порівняйте результат із моделлю, що використовує повний набір ознак.

*Технічна примітка:* Використайте `mutual_info_classif` з `SelectKBest` у `sklearn.feature_selection` для відбору найбільш значущих ознак перед реалізуванням `RandomForestClassifier`.

---

**Варіант 5:** Реалізуйте побудову нових ознак через комбінацію поліноміальних взаємодій. Використайте `PolynomialFeatures` для створення взаємодій між основними ознаками, а потім побудуйте модель випадкового лісу на новому наборі ознак. Проаналізуйте, як додавання поліноміальних взаємодій впливає на результативність моделі.

*Технічна примітка:* Використайте `PolynomialFeatures(degree=2, interaction_only=True)` з `sklearn.preprocessing` для створення взаємодій перед реалізуванням `RandomForestClassifier`.

---

**Варіант 6:** Використайте метод відбору ознак на основі об’єднання показників кореляції та важливості ознак у моделі випадкового лісу. Спершу виберіть ознаки, що мають високу кореляцію із цільовою змінною, а потім Використайте `SelectFromModel` для подальшого відбору. Побудуйте остаточну модель на основі цих обраних ознак.

*Технічна примітка:* Використайте `data.corr()` з бібліотеки `pandas` для обчислення кореляції та `SelectFromModel(RandomForestClassifier())` для подальшого відбору ознак.

---

**Варіант 7:** Реалізуйте кластеризацію ознак за допомогою `FeatureAgglomeration` для групування ознак зі схожими характеристиками. Побудуйте нову модель випадкового лісу на основі зменшеного набору ознак і проаналізуйте, як кластеризація впливає на результативність і стабільність моделі.

*Технічна примітка:* Використайте `FeatureAgglomeration(n_clusters=10)` з `sklearn.cluster` для кластеризації ознак перед реалізуванням `RandomForestClassifier`.

---

**Варіант 8:** Створіть нові ознаки, розрахувавши середні значення, медіану та стандартне відхилення для груп споріднених ознак. Побудуйте модель випадкового лісу на розширеному наборі ознак та оцініть, як додавання статистичних характеристик впливає на результативність передбачень.

*Технічна примітка:* Використайте `mean()`, `median()` та `std()` з бібліотеки `pandas` для обчислення нових ознак і `RandomForestClassifier` для моделювання.

---

**Варіант 9:** Виконайте відбір ознак на основі їхньої ваги в моделі випадкового лісу, зберігаючи тільки ті ознаки, які мають вагу понад певний поріг. Побудуйте остаточну модель на основі обраних ознак і порівняйте її точність із початковою моделлю.

*Технічна примітка:* Використайте `feature_importances_` у `RandomForestClassifier` для визначення ваг ознак і `SelectFromModel` для їхнього відбору.

---

**Варіант 10:** Виконайте перетворення вхідних ознак за допомогою `QuantileTransformer`, щоби привести їх до нормального розподілу. Порівняйте точність моделі випадкового лісу, побудованої на перетворених ознаках, з моделлю, що використовує початкові ознаки, для оцінювання впливу нормалізації на результативність.

*Технічна примітка:* Використайте `QuantileTransformer(output_distribution='normal')` з `sklearn.preprocessing` перед реалізуванням `RandomForestClassifier`.

---

**Варіант 11:** Використайте `VarianceThreshold` для відбору ознак із високою варіативністю, щоб виключити ознаки з низькою інформаційною цінністю. Це дасть змогу зменшити розмірність даних і зосередитися на найбільш значущих ознаках, перш ніж будувати модель випадкового лісу.

*Технічна примітка:* Використайте `VarianceThreshold(threshold=0.1)` з `sklearn.feature_selection` для відбору ознак, які мають варіативність понад вказаний поріг, а потім побудуйте модель з `RandomForestClassifier`.

---

**Варіант 12:** Застосуйте метод відбору ознак `Lasso` до моделі випадкового лісу перед її побудовою, щоб виключити менш значущі ознаки. Відібрані ознаки, зокрема з ненульовими ваговими коефіцієнтами, будуть використовуватися для подальшого аналізу й порівняння результативності з повною моделлю.

*Технічна примітка:* Використайте `Lasso(alpha=0.01)` з `SelectFromModel` у `sklearn.feature_selection` для відбору ознак перед реалізуванням `RandomForestClassifier`.

---

**Варіант 13:** Використайте `SelectFpr` для відбору ознак із високим рівнем статистичної значущості (за рівнем похибки першого роду). Оберіть найважливіші ознаки для подальшого моделювання та порівняйте точність передбачень із початковою моделлю.

*Технічна примітка:* Використайте `SelectFpr(score_func=f_classif, alpha=0.05)` з `sklearn.feature_selection` для відбору значущих ознак перед побудовою моделі з `RandomForestClassifier`.

---

**Варіант 14:** Використайте комбінацію відбору ознак на основі важливості (важливість перестановок) та статистичних показників. Спершу використайте `SelectKBest` для визначення найінформативніших ознак, а потім підтвердіть їх важливість через `permutation_importance` для моделі випадкового лісу.

*Технічна примітка:* Використайте `SelectKBest(score_func=f_classif, k=10)` та `permutation_importance` з `sklearn.inspection` перед остаточним реалізуванням `RandomForestClassifier`.

---

**Варіант 15:** Створіть нові ознаки, об’єднуючи початкові ознаки в пари через добуток значень. Це дасть змогу побудувати нові нелінійні взаємозв’язки, які можуть бути значущими для моделі випадкового лісу. Порівняйте результативність нової моделі з оригінальною.

*Технічна примітка:* Використайте `PolynomialFeatures(degree=2, include_bias=False)` з `sklearn.preprocessing` для створення добутків значень ознак перед реалізуванням `RandomForestClassifier`.

---

**Варіант 16:** Використайте метод `Univariate Feature Selection`, щоби зберегти тільки ознаки з найвищим показником значущості. Застосуйте критерій `chi2` до визначення найінформативніших ознак, а потім побудуйте модель випадкового лісу на відфільтрованих ознаках.

*Технічна примітка:* Використайте `SelectKBest(score_func=chi2, k=10)` з `sklearn.feature_selection` для відбору ознак перед реалізуванням `RandomForestClassifier`.

---

**Варіант 17:** Виконайте відбір ознак на основі `Ridge Regression`, щоби зменшити кількість ознак, зберігаючи тільки ті, які найбільше впливають на цільову змінну. Порівняйте результат моделі випадкового лісу з обраними ознаками та з повним набором ознак.

*Технічна примітка:* Використайте `Ridge(alpha=1.0)` з `SelectFromModel` для відбору ознак, а потім реалізуйте `RandomForestClassifier`.

---

**Варіант 18:** Створіть комбінації ознак на основі кластеризації, застосувавши `KMeans` до початкових ознак. Використайте результуючі кластери як нові ознаки та побудуйте модель випадкового лісу на основі зменшеного набору, щоб оцінити, як групування ознак впливає на точність передбачень.

*Технічна примітка:* Використайте `KMeans(n_clusters=5)` з `sklearn.cluster` для створення кластерів, а потім `RandomForestClassifier` для побудови моделі.

---

**Варіант 19:** Дослідіть вплив нормалізації ознак, застосовуючи до них `StandardScaler`, `RobustScaler` та `MinMaxScaler`, перед побудовою моделі випадкового лісу. Порівняйте точність і стабільність передбачень для кожного типу нормалізації, щоби зрозуміти, яка з них найбільше підходить для вашого набору даних.

*Технічна примітка:* Використайте `StandardScaler()`, `RobustScaler()`, та `MinMaxScaler()` з `sklearn.preprocessing`, застосовуючи до них `RandomForestClassifier`.

---

**Варіант 20:** Використайте метод відбору ознак на основі `Maximal Information Coefficient (MIC)` для визначення ознак із високим рівнем нелінійної залежності із цільовою змінною. Використайте відібрані ознаки для побудови моделі випадкового лісу та порівняйте точність із моделлю на початковому наборі ознак.

*Технічна примітка:* Використайте бібліотеку `minepy` для обчислення MIC та відбору ознак перед реалізуванням `RandomForestClassifier`.

<a class="anchor" id="lab-6.6"></a>

## <span style="color:blue; font-size:1.2em;">6.6. Покращення та подальший тюнинг моделей</span>

[Повернутися до змісту](#lab-6)

### <span style="color:red; font-size:1.5em;">Завдання 5</span>

---

**Варіант 1:** Використайте метод `Gradient Boosting` для побудови ансамблевої моделі та виконайте відбір ознак на основі важливості, наданої цією моделлю. Збережіть ознаки з високим ступенем важливості та порівняйте результативність моделі, побудованої на відібраних ознаках, з моделлю на повному наборі ознак.

*Технічна примітка:* Використайте `GradientBoostingClassifier` із параметром `n_estimators=100`, а потім `feature_importances_` для відбору ознак.

---

**Варіант 2:** Проведіть `AdaBoost` із базовими моделями, такими як дерева рішень або логістична регресія, та виконайте трансформацію ознак на основі важливості, яку надає ця модель. Це дасть можливість зрозуміти, як зберігати тільки впливові ознаки, що значно впливають на точність передбачень у цій моделі.

*Технічна примітка:* Використайте `AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1))` з `feature_importances_` для визначення важливих ознак.

---

**Варіант 3:** Застосуйте метод багатовимірного шкалювання (`MDS`) до перетворення ознак у простір меншої розмірності перед побудовою моделі. Це дасть можливість краще зрозуміти структуру даних та оцінити, як зменшення розмірності впливає на результативність моделі ансамблю.

*Технічна примітка:* Використайте `MDS(n_components=5)` з `sklearn.manifold`, щоб зменшити розмірність, а потім побудуйте модель із `RandomForestClassifier`.

---

**Варіант 4:** Виконайте кластеризацію ознак за допомогою `Agglomerative Clustering`, щоб згрупувати ознаки з подібними характеристиками, а потім створіть нові групові ознаки. Побудуйте модель ансамблю на основі цих нових груп і проаналізуйте, чи зберігаються значущі зв’язки та, чи покращується точність.

*Технічна примітка:* Використайте `AgglomerativeClustering(n_clusters=10)` з `sklearn.cluster`, щоб створити нові групи ознак перед реалізуванням `GradientBoostingClassifier`.

---

**Варіант 5:** Виконайте відбір ознак за допомогою `Recursive Feature Elimination (RFE)` на основі моделі `Gradient Boosting`. Це дасть можливість залишити лише найбільш важливі ознаки, що сприяють покращенню точності передбачень. Порівняйте результати моделі до та після відбору ознак.

*Технічна примітка:* Використайте `RFE(estimator=GradientBoostingClassifier(), n_features_to_select=10)` з `sklearn.feature_selection`.

---

**Варіант 6:** Реалізуйте трансформацію ознак через метод `Isomap` для збереження нелінійних зв’язків між ознаками, зменшуючи водночас їхню розмірність. Це дасть можливість зберегти основну інформацію та побудувати компактнішу модель для побудови ансамблю.

*Технічна примітка:* Використайте `Isomap(n_components=5)` з `sklearn.manifold`, щоб зменшити розмірність ознак, а потім реалізуйте `AdaBoostClassifier`.

---

**Варіант 7:** Застосуйте `SelectFromModel` з моделлю `Random Forest` до відбору найважливіших ознак, що сприяють точності передбачення. Використайте модель із відфільтрованими ознаками для побудови іншого ансамблю, наприклад, `Gradient Boosting`, і порівняйте результати з повною моделлю.

*Технічна примітка:* Використайте `SelectFromModel(estimator=RandomForestClassifier())` з `sklearn.feature_selection`, щоб відсіяти маловпливові ознаки, та `GradientBoostingClassifier` для подальшого моделювання.

---

**Варіант 8:** Виконайте поліноміальну трансформацію ознак другого ступеня, зберігаючи тільки найвпливовіші поліноміальні взаємодії. Побудуйте модель ансамблю, наприклад, `AdaBoost`, на новому наборі ознак та порівняйте результати з моделлю, побудованою на оригінальних ознаках.

*Технічна примітка:* Використайте `PolynomialFeatures(degree=2)` з `sklearn.preprocessing` для створення поліноміальних взаємодій перед реалізуванням `AdaBoostClassifier`.

---

**Варіант 9:** Виконайте об’єднання ознак через `FeatureUnion` для комбінації результатів декількох перетворень (наприклад, PCA та Kernel PCA). Застосуйте ансамблеву модель до нових комбінацій ознак, оцінюючи, як це впливає на результативність моделі.

*Технічна примітка:* Використайте `FeatureUnion([('pca', PCA(n_components=5)), ('kpca', KernelPCA(kernel='rbf', n_components=5))])` з `sklearn.pipeline` для створення нових об’єднаних ознак перед реалізуванням `RandomForestClassifier`.

---

**Варіант 10:** Виконайте `Mutual Information Feature Selection` для оцінювання ознак, що найкраще пояснюють цільову змінну, а потім побудуйте модель ансамблю на основі відібраних ознак. Проаналізуйте, як залишення тільки ознак із високою взаємною інформацією покращує точність передбачення.

*Технічна примітка:* Використайте `mutual_info_classif` з `SelectKBest` для відбору ознак перед реалізуванням `GradientBoostingClassifier`.

---

**Варіант 11:** Виконайте `Partial Least Squares (PLS)` для перетворення набору ознак, щоби зменшити кількість ознак та зберегти основні взаємозв’язки із цільовою змінною. Використайте новий набір зменшених ознак для побудови моделі `AdaBoost` і порівняйте її точність із моделлю на початкових ознаках.

*Технічна примітка:* Використайте `PLSRegression(n_components=10)` з `sklearn.cross_decomposition` для перетворення ознак перед реалізуванням `AdaBoostClassifier`.

---

**Варіант 12:** Здійсніть `Quantile Transformation` ознак, аби зменшити вплив викривлень розподілу та зробити дані більш нормально розподіленими. Використайте отримані ознаки для побудови моделі `Gradient Boosting` і проаналізуйте, чи впливає таке перетворення на стабільність передбачення.

*Технічна примітка:* Використайте `QuantileTransformer(output_distribution='normal')` з `sklearn.preprocessing`, а потім реалізуйте `GradientBoostingClassifier`.

---

**Варіант 13:** Використайте `SelectPercentile`, щоб відібрати відсоток найважливіших ознак за їхньою значущістю для цільової змінної. Зберігши, наприклад, 20 % найкращих ознак, побудуйте модель `Random Forest` і порівняйте точність із моделлю на повному наборі ознак.

*Технічна примітка:* Використайте `SelectPercentile(score_func=f_classif, percentile=20)` з `sklearn.feature_selection` для відбору ознак, а потім реалізуйте `RandomForestClassifier`.

---

**Варіант 14:** Використайте метод `Kernel PCA` для створення нелінійних компонент, що зменшить розмірність і може підвищити точність моделі. Побудуйте ансамблеву модель, наприклад, `AdaBoost`, на новому наборі ознак і порівняйте її результативність із моделлю на початкових ознаках.

*Технічна примітка:* Використайте `KernelPCA(kernel='rbf', n_components=5)` з `sklearn.decomposition` для створення нових компонент перед реалізуванням `AdaBoostClassifier`.

---

**Варіант 15:** Застосуйте метод `Stability Selection` до відбору ознак на основі їхньої стабільності на різних випадкових вибірках. Побудуйте модель `Gradient Boosting` на відібраних ознаках і порівняйте її з моделлю, побудованою на повному наборі, щоб оцінити стабільність передбачень.

*Технічна примітка:* Використайте `StabilitySelection` з `sklearn.linear_model` разом із `GradientBoostingClassifier` для порівняння моделей на основі стабільності ознак.

---

**Варіант 16:** Використайте `MaxAbsScaler`, щоб нормалізувати ознаки до діапазону [-1, 1] та зберегти їхні пропорційні значення. Порівняйте результативність моделі `Random Forest` на нормалізованих ознаках із результативністю на початкових ознаках.

*Технічна примітка:* Використайте `MaxAbsScaler()` з `sklearn.preprocessing` для нормалізації ознак перед реалізуванням `RandomForestClassifier`.

---

**Варіант 17:** Використайте алгоритм `Feature Elimination by Shuffling` для відбору ознак через випадкову перестановку кожної ознаки. Визначте ознаки, які мають найбільший вплив на точність моделі `Gradient Boosting`, та виключіть менш значущі ознаки.

*Технічна примітка:* Використайте `permutation_importance` з `sklearn.inspection` для визначення значущих ознак, а потім реалізуйте `GradientBoostingClassifier`.

---

**Варіант 18:** Проведіть перетворення `Box-Cox Transformation` на ознаках, що мають позитивні значення, для зменшення їхньої асиметрії. Побудуйте модель `AdaBoost` на перетворених ознаках і проаналізуйте, як така трансформація впливає на результативність.

*Технічна примітка:* Використайте `scipy.stats.boxcox` для трансформації ознак, що мають позитивні значення, перед реалізуванням `AdaBoostClassifier`.

---

**Варіант 19:** Виконайте `Feature Embedding` на категоріальних ознаках за допомогою нейронної мережі, щоби створити нові вбудовані представлення. Побудуйте ансамблеву модель `Random Forest` на новому наборі вбудованих ознак та оцініть, чи підвищує це точність порівняно з one-hot кодуванням.

*Технічна примітка:* Використайте бібліотеку `Keras` для створення вбудованих представлень категоріальних ознак, а потім застосуйте `RandomForestClassifier` до моделювання.

---

**Варіант 20:** Проведіть `Target Encoding` на категоріальних ознаках для отримання числових представлень на основі середнього значення цільової змінної для кожної категорії. Це дасть можливість створити кращі числові ознаки для побудови моделі `Gradient Boosting`.

*Технічна примітка:* Використайте `TargetEncoder` з `category_encoders` для оброблення категоріальних ознак перед реалізуванням `GradientBoostingClassifier`.